In [1]:
%pip install -q ultralytics deep_sort_realtime opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import matplotlib.pyplot as plt

# 2. Load video
video_path = "../utilities/cars_on_the_highway.mp4"
if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video file {video_path} not found. Please check the path.")

cap = cv2.VideoCapture(video_path)

# 3. Load YOLO model for detection
model_path = "../utilities/yolov8n.pt"  # Path to your YOLO model
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file {model_path} not found. Please check the path.")

model = YOLO(model_path)

# 4. Deep SORT inicializálása
tracker = DeepSort(max_age=30)

# 5. Tracking adatok tárolására lista
tracked_data = []
frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame, classes=[2, 5, 7], conf=0.4, verbose=False)
    detections = []

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].cpu().item()
            cls = int(box.cls[0].item())
            detections.append([[x1, y1, x2 - x1, y2 - y1], conf, str(cls)])

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)

        tracked_data.append([frame_idx, track_id, x1, y1, x2, y2])

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    frame_idx += 1
    # cv2.namedWindow("Tracking", cv2.WINDOW_NORMAL)
    # cv2.resizeWindow("Tracking", 800, 600)
    # cv2.imshow("Deep SORT Tracking", frame)
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break

cap.release()
cv2.destroyAllWindows()

# 6. Eredmények mentése CSV-be
columns = ["frame", "id", "x1", "y1", "x2", "y2"]
df = pd.DataFrame(tracked_data, columns=columns)
df.to_csv("trajectory_data_deepsort.csv", index=False)
print("Tracking data saved to trajectory_data_deepsort.csv")


Tracking data saved to trajectory_data_deepsort.csv
